In [2]:
from time import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans
from sklearn.mixture import GaussianMixture as EM
from sklearn.decomposition import PCA
from sklearn import datasets
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler, scale
from sklearn.metrics import silhouette_samples, silhouette_score
import matplotlib.cm as cm
from matplotlib.ticker import MaxNLocator

def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

np.random.seed(5)

def processed_data(name, X, y):
    X_raw = X
    y_raw =y
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=111, stratify=y)

    # Preprocessing the data between 0 and 1
    scaler = preprocessing.MinMaxScaler(feature_range=[0,1]).fit(X)
#     X_train = scaler.transform(X_train)
#     X_test = scaler.transform(X_test)
    X = scaler.transform(X)

    
    n_samples, n_features = X.shape
    n_targets = len(np.unique(y))
    
    return {
        'name':name,
        'X_raw': X_raw,
        'y_raw': y_raw,
        'X':X,
        'y':y,
        'X_train':X_train, 
        'X_test': X_test, 
        'y_train': y_train, 
        'y_test': y_test,
        'n_samples': n_samples,
        'n_features': n_features,
        'n_targets': n_targets
    }

def credit_data():
    data=np.genfromtxt('../dataset/german.data-numeric.txt') 
    X,y = data[:,:-1], data[:,-1:].squeeze() 
    return processed_data('credit-g',X, y)

def banknote_data():
    data=np.genfromtxt('../dataset/data_banknote_authentication.txt', delimiter=',') 
    X,y = data[:,:-1], data[:,-1:].squeeze() 
    return processed_data('banknote',X,y)

def wine_data():
    data=np.genfromtxt('../dataset/winequality-white.csv', delimiter=';', skip_header=1)
    X,y = data[:,:-1], data[:,-1:].squeeze()
    return processed_data('wine-quality',X,y)
    
# def wine_data():
#     data=np.genfromtxt('../dataset/winequality.csv', delimiter=';')
#     X,y = data[:,:-1], data[:,-1:].squeeze()
#     return processed_data('wine-quality',X,y)

def iris_data():
    from sklearn.datasets import load_iris
    data = load_iris()
    X,y = data.data, data.target
    return processed_data('iris',X,y)


cdata = credit_data()
bdata = banknote_data()
idata = iris_data()
wdata = wine_data()

########################################################################


   
def bench_k_means(estimator, name, data, labels, sample_size):
    t0 = time()
    estimator.fit(data)
    
    cluster_labels = estimator.labels_
        
    return {
        'init': name,
        'time': (time() - t0),
        'accuracy': metrics.accuracy_score(labels, cluster_labels),
        'precision': metrics.precision_score(labels, cluster_labels, average="weighted"),
        'recall': metrics.recall_score(labels, cluster_labels, average="weighted"),
        'f1_score': metrics.f1_score(labels, cluster_labels, average="weighted"),
        'homogenity': metrics.homogeneity_score(labels, cluster_labels),
        'completeness': metrics.completeness_score(labels, cluster_labels),
        'v-measure': metrics.v_measure_score(labels, cluster_labels),
        'ARI': metrics.adjusted_rand_score(labels, cluster_labels),
        'AMI': metrics.adjusted_mutual_info_score(labels,  cluster_labels),
        'mutual_info': metrics.mutual_info_score(labels,  cluster_labels),
        'NMI': metrics.cluster.normalized_mutual_info_score(labels, cluster_labels),
        'silhouette': metrics.silhouette_score(data, cluster_labels)
    }

def get_scores(name, data, labels, cluster_labels):
    return {
        'init': name,
        'accuracy': metrics.accuracy_score(labels, cluster_labels),
        'precision': metrics.precision_score(labels, cluster_labels, average="weighted"),
        'recall': metrics.recall_score(labels, cluster_labels, average="weighted"),
        'f1_score': metrics.f1_score(labels, cluster_labels, average="weighted"),
        'homogenity': metrics.homogeneity_score(labels, cluster_labels),
        'completeness': metrics.completeness_score(labels, cluster_labels),
        'v-measure': metrics.v_measure_score(labels, cluster_labels),
        'ARI': metrics.adjusted_rand_score(labels, cluster_labels),
        'AMI': metrics.adjusted_mutual_info_score(labels,  cluster_labels),
        'mutual_info': metrics.mutual_info_score(labels,  cluster_labels),
        'NMI': metrics.cluster.normalized_mutual_info_score(labels, cluster_labels),
        'silhouette': metrics.silhouette_score(data, cluster_labels)
    }

def performance_k_means(data, labels, dataset_name, n_clusters_range):
    vals = []
    for n in n_clusters_range:
        estimator = KMeans(init='k-means++', 
                           n_clusters=n, 
                           max_iter = 300, 
                           n_init = 10, 
                           random_state = 0)
        
        t0 = time()
        estimator.fit(data)
        _time = (time() - t0)
        
        cluster_labels = estimator.labels_
        
        _bench = get_scores("k-means++", 
                                     data, 
                                     labels, 
                                     cluster_labels)
        _bench['inertia'] = estimator.inertia_
        _bench['n_clusters'] = n
        _bench['time'] = _time
        vals.append(_bench)

    df = pd.DataFrame(vals)
    return df

def performance_em(data, labels, dataset_name, n_clusters_range, covariance_type="spherical"):
    vals = []
    for n in n_clusters_range:
        estimator = EM(n_components=n,
                       covariance_type=covariance_type,
                       n_init=10,
                       warm_start=True,
                       random_state=0,
                       init_params='kmeans')
        
        t0 = time()
        estimator.fit(data)
        _time = (time() - t0)
        
        cluster_labels = estimator.predict(data)
        
        _bench = get_scores("gmm-em", 
                                     data, 
                                     labels, 
                                     cluster_labels)
#         _bench['inertia'] = estimator.inertia_
        _bench['aic'] = estimator.aic(data)
        _bench['bic'] = estimator.bic(data)
        _bench['score'] = estimator.score(data)
        _bench['n_clusters'] = n
        _bench['time'] = _time
        vals.append(_bench)

    df = pd.DataFrame(vals)
    return df


def k_means_performance_curve(dataset_name,n_clusters_range, data, labels):
    n_samples, n_features = data.shape
    n_classes = len(np.unique(labels))
    sample_size = n_samples

    vals = []
    for n in n_clusters_range:
        estimator = KMeans(init='k-means++', n_clusters=n, max_iter = 300, n_init = 10, random_state = 0)
        _bench = bench_k_means(estimator, name="k-means++", data=data, labels=labels, sample_size=sample_size)
        _bench['n_clusters'] = n
        vals.append(_bench)

    df = pd.DataFrame(vals)    
    ax1 = df.plot(x='n_clusters', 
                  y=['silhouette'], 
                  title="K Means Silhoutte Score %s"%dataset_name)
    ax1.set_xlabel("Number of clusters")
    plt.show()
        
    df = pd.DataFrame(vals)    
    ax1 = df.plot(x='n_clusters', 
                  y=['AMI','silhouette'], 
                  title="K Means Performance evaluation %s"%dataset_name)
    ax1.set_xlabel("Number of clusters")
    plt.show()

def bench_em(estimator, name, data, labels, sample_size):
    t0 = time()
    estimator.fit(data)
    
    cluster_labels = estimator.predict(data)
        
    return {
        'init': name,
        'time': (time() - t0),
        'accuracy': metrics.accuracy_score(labels, cluster_labels),
        'precision': metrics.precision_score(labels, cluster_labels, average="weighted"),
        'recall': metrics.recall_score(labels, cluster_labels, average="weighted"),
        'f1_score': metrics.f1_score(labels, cluster_labels, average="weighted"),
        'homogenity': metrics.homogeneity_score(labels, cluster_labels),
        'completeness': metrics.completeness_score(labels, cluster_labels),
        'v-measure': metrics.v_measure_score(labels, cluster_labels),
        'ARI': metrics.adjusted_rand_score(labels, cluster_labels),
        'AMI': metrics.adjusted_mutual_info_score(labels,  cluster_labels),
        'mutual_info': metrics.mutual_info_score(labels,  cluster_labels),
        'NMI': metrics.cluster.normalized_mutual_info_score(labels, cluster_labels),
        'silhouette': metrics.silhouette_score(data, cluster_labels),
        'aic': estimator.aic(data),
        'bic': estimator.bic(data)
    }
    
def em_performance_curve(dataset_name,n_clusters_range, data, labels,cov_type='spherical'):
    n_samples, n_features = data.shape
    n_classes = len(np.unique(labels))
    sample_size = n_samples

    vals = []
    for n in n_clusters_range:
        estimator = EM(n_components=n,covariance_type=cov_type,n_init=1,warm_start=True,random_state=100,init_params='kmeans')
        _bench = bench_em(estimator, name="em", data=data, labels=labels, sample_size=sample_size)
        _bench['n_clusters'] = n
        vals.append(_bench)
        
    fig, (ax1, ax2) = plt.subplots(1, 2)
    fig.set_size_inches(18, 7)
        
    
    df = pd.DataFrame(vals)    
    df.plot(x='n_clusters', ax=ax1,
              y=['silhouette'], 
              title="GMM Silhoutte Score %s"%dataset_name)
    ax1.set_xlabel("Number of components")

        
    df = pd.DataFrame(vals)    
    df.plot(x='n_clusters', ax=ax2,
                  y=['bic','aic'], 
                  title="BIC and AIC Scores %s"%dataset_name)
    ax2.set_xlabel("Number of components")
    plt.show()
    plt.close()
    
    fig2, (axs) = plt.subplots(1, 1, constrained_layout=True)
    
    grads = np.gradient(df['bic'].tolist())
    axs.plot(df['n_clusters'].tolist(),grads)
    axs.set_title("Gradient of BIC Scores %s"%dataset_name)
    axs.set_xlabel("Number of components")
    plt.show()
    
    
def silhouette_plot(dataset_name, range_n_clusters, data):
    for n_clusters in range_n_clusters:
        # Create a subplot with 1 row and 2 columns
        fig, (ax1, ax2) = plt.subplots(1, 2)
        fig.set_size_inches(18, 7)

        # The 1st subplot is the silhouette plot
        # The silhouette coefficient can range from -1, 1 but in this example all
        # lie within [-0.1, 1]
        ax1.set_xlim([-0.1, 1])
        # The (n_clusters+1)*10 is for inserting blank space between silhouette
        # plots of individual clusters, to demarcate them clearly.
        ax1.set_ylim([0, len(data) + (n_clusters + 1) * 10])

        # Initialize the clusterer with n_clusters value and a random generator
        # seed of 10 for reproducibility.
        clusterer = KMeans(n_clusters=n_clusters, random_state=10)
        cluster_labels = clusterer.fit_predict(data)

        # The silhouette_score gives the average value for all the samples.
        # This gives a perspective into the density and separation of the formed
        # clusters
        silhouette_avg = silhouette_score(data, cluster_labels)
        print("For n_clusters =", n_clusters,
              "The average silhouette_score is :", silhouette_avg)

        # Compute the silhouette scores for each sample
        sample_silhouette_values = silhouette_samples(data, cluster_labels)

        y_lower = 10
        for i in range(n_clusters):
            # Aggregate the silhouette scores for samples belonging to
            # cluster i, and sort them
            ith_cluster_silhouette_values = \
                sample_silhouette_values[cluster_labels == i]

            ith_cluster_silhouette_values.sort()

            size_cluster_i = ith_cluster_silhouette_values.shape[0]
            y_upper = y_lower + size_cluster_i

            color = cm.nipy_spectral(float(i) / n_clusters)
            ax1.fill_betweenx(np.arange(y_lower, y_upper),
                              0, ith_cluster_silhouette_values,
                              facecolor=color, edgecolor=color, alpha=0.7)

            # Label the silhouette plots with their cluster numbers at the middle
            ax1.text(-0.05, y_lower + 0.5 * size_cluster_i, str(i))

            # Compute the new y_lower for next plot
            y_lower = y_upper + 10  # 10 for the 0 samples

        ax1.set_title("The silhouette plot for %s n_clusters=%s"%(dataset_name,n_clusters))
        ax1.set_xlabel("The silhouette coefficient values")
        ax1.set_ylabel("Cluster label")

        # The vertical line for average silhouette score of all the values
        ax1.axvline(x=silhouette_avg, color="red", linestyle="--")

        ax1.set_yticks([])  # Clear the yaxis labels / ticks
        ax1.set_xticks([-0.1, 0, 0.2, 0.4, 0.6, 0.8, 1])

        # 2nd Plot showing the actual clusters formed
        colors = cm.nipy_spectral(cluster_labels.astype(float) / n_clusters)
        ax2.scatter(data[:, 0], data[:, 1], marker='.', s=30, lw=0, alpha=0.7,
                    c=colors, edgecolor='k')

        # Labeling the clusters
        centers = clusterer.cluster_centers_
        # Draw white circles at cluster centers
        ax2.scatter(centers[:, 0], centers[:, 1], marker='o',
                    c="white", alpha=1, s=200, edgecolor='k')

        for i, c in enumerate(centers):
            ax2.scatter(c[0], c[1], marker='$%d$' % i, alpha=1,
                        s=50, edgecolor='k')

        ax2.set_title("The visualization of the clustered data.")
        ax2.set_xlabel("Feature space for the 1st feature")
        ax2.set_ylabel("Feature space for the 2nd feature")

        plt.suptitle(("Silhouette analysis for KMeans clustering on %s dataset "
                      "with n_clusters = %d" % (dataset_name,n_clusters)),
                     fontsize=14, fontweight='bold')

    plt.show()

def pca_scatter_two_component(dataset_name, X, y):
    pca = PCA(n_components=2)
    principal_components = pca.fit_transform(X)
    v = pca.explained_variance_ratio_
    principal_df = pd.DataFrame(data = principal_components, columns = ['pc1','pc2'])
    final_df = pd.concat([principal_df, pd.DataFrame({'target': y})],axis=1)
    
    fig, ax = plt.subplots()
    final_df.plot.scatter(x='pc1', y='pc2', c='target', colormap='rainbow', s=10, ax=ax)
    ax.set_xlabel("pc1 (%0.2f)"%v[0])
    ax.set_ylabel("pc2 (%0.2f)"%v[1])
    ax.set_title('2 Component PCA for %s dataset'%dataset_name)
    plt.show()
    

def pca_components(dataset_name, X, y, percentage):
    pca = PCA(percentage)
    principal_components = pca.fit_transform(X)
    v = pca.explained_variance_ratio_
    return {
        'dataset': dataset_name,
        'variance': percentage,
        'principle_components': len(v),
        'variances': v
    }
    
def pca_performance(data):
    ydata_kmeans = []
    ydata_em = []
    xdata = list(range(1,data['n_features']+1))
    
    kmeans_orig = KMeans(init='k-means++', n_clusters=data['n_targets'], n_init=10)
    kmeans_orig.fit(data['X'])
    kmeans_orig_cluster_labels = kmeans_orig.labels_
    ydata_kmeans_orig = metrics.silhouette_score(data['X'], kmeans_orig_cluster_labels)
    
    em_orig = EM(n_components=data['n_targets'],covariance_type="spherical",n_init=1,warm_start=True,random_state=100,init_params='kmeans')
    em_orig.fit(data['X'])
    em_orig_cluster_labels = em_orig.predict(data['X'])
    ydata_em_orig = metrics.silhouette_score(data['X'], em_orig_cluster_labels)
    
    for i in xdata:
        reduced_data = PCA(n_components=i).fit_transform(data['X'])
        kmeans = KMeans(init='k-means++', n_clusters=data['n_targets'], n_init=10)
        kmeans.fit(reduced_data)
        kmeans_cluster_labels = kmeans.labels_
        ydata_kmeans.append(metrics.silhouette_score(reduced_data, kmeans_cluster_labels))
        
        em = EM(n_components=data['n_targets'],covariance_type="spherical",n_init=1,warm_start=True,random_state=100,init_params='kmeans')
        em.fit(reduced_data)
        em_cluster_labels = em.predict(reduced_data)
        ydata_em.append(metrics.silhouette_score(reduced_data, em_cluster_labels))
        

    #Plotting the results onto a line graph, allowing us to observe 'The elbow'
    plt.plot(xdata, ydata_kmeans, label="K Means")
    plt.plot(xdata, ydata_em, label="EM")
    plt.title('Performance PCA reduced (%s)'%data['name'])
    plt.xlabel('Principal components')
    plt.ylabel('Silhoutte Score') #within cluster sum of squares
    plt.legend()
    plt.show()
    
def two_component_figures(data):
    np.random.seed(5)
    pca = PCA(n_components=2)
    pca.fit(data['X'])
    X = pca.transform(data['X'])
    v = pca.explained_variance_ratio_
    pl.scatter(X[:,0], X[:, 1], c=data['y'], cmap='rainbow')
    pl.xlabel("pc1 (%0.2f)"%v[0])
    pl.ylabel("pc2 (%0.2f)"%v[1])
    pl.title("2 Components PCA %s"%data['name'])
    pl.legend()
    pl.show()

    ica = ICA(n_components=2)
    ica.fit(data['X'])
    X = ica.transform(data['X'])
    pl.scatter(X[:,0], X[:, 1], c=data['y'], cmap='rainbow')
    pl.title("2 Components ICA %s"%data['name'])
    pl.show()

    rca = RCA(n_components=2)
    rca.fit(data['X'])
    X = rca.transform(data['X'])
    pl.scatter(X[:,0], X[:, 1], c=data['y'], cmap='rainbow')
    pl.title("2 Components RCA %s"%data['name'])
    pl.show()

    lda = LDA(n_components=2)
    lda.fit(data['X'], data['y'])
    X = lda.transform(data['X'])
    pl.scatter(X[:,0],X[:, 1], c=data['y'], cmap='rainbow')
    pl.title("2 Components LDA %s"%data['name'])
    pl.show()

def reduced_data_k_means_perf(data, reduced_data, n_clusters_range, method="ICA", ncluster=None):
    dfo = performance_k_means(data['X'], 
                                  data['y'], 
                                  data['name'], 
                                  n_clusters_range=n_clusters_range)

    df = performance_k_means(reduced_data, 
                                  data['y'], 
                                  data['name'], 
                                  n_clusters_range=n_clusters_range)

    from matplotlib.ticker import MaxNLocator
    print(np.argmax(df['AMI']))
    print(np.argmax(df['silhouette']))

    fig, (ax1, ax2) = plt.subplots(1, 2)
    fig.set_size_inches(18, 7)

    ax1.plot(dfo['n_clusters'], dfo['AMI'],label="AMI")
    ax1.plot(df['n_clusters'], df['AMI'],label="AMI "+method)
    ax1.plot(dfo['n_clusters'], dfo['silhouette'],label="silhouette")
    ax1.plot(df['n_clusters'], df['silhouette'],label="silhouette "+method)
    ax1.set_title("K Means Performance evaluation %s"%data['name'])
    ax1.set_xlabel("Number of clusters")
    ax1.set_ylabel("Scores")
    ax1.legend()

    ax2.plot(dfo['n_clusters'], dfo['inertia'],label="inertia")
    ax2.plot(df['n_clusters'], df['inertia'],label="inertia "+method)
    ax2.set_title("K Means Inertia %s"%data['name'])
    ax2.set_xlabel("Number of clusters")
    ax2.set_ylabel("Scores")
    ax2.legend()


    fig, ax3 = plt.subplots(1,1)
    ax3.plot(df['n_clusters'], df['time'], label="time")
    ax3.plot(dfo['n_clusters'], dfo['time'], label="time "+method)
    ax3.set_title("K Means Time %s"%data['name'])
    ax3.legend()

    plt.show()

    if not ncluster:
        ncluster = data['n_targets']
        
    perfo = dfo[df['n_clusters']==ncluster]
    perf = df[df['n_clusters']==ncluster]

    print(perfo)
    print("------")
    print(perf)
    

def reduced_data_em_perf(data, reduced_data, n_clusters_range, method="ICA", ncluster=None):
    dfo = performance_em(data['X'], 
                                  data['y'], 
                                  data['name'], 
                                  n_clusters_range=n_clusters_range)

    df = performance_em(reduced_data, 
                          data['y'], 
                          data['name'], 
                          n_clusters_range=n_clusters_range)

    from matplotlib.ticker import MaxNLocator
    print(np.argmax(df['AMI']))
    print(np.argmax(df['silhouette']))

    fig, (ax1, ax2) = plt.subplots(1, 2)
    fig.set_size_inches(18, 7)

    ax1.plot(dfo['n_clusters'], dfo['AMI'],label="AMI")
    ax1.plot(df['n_clusters'], df['AMI'],label="AMI "+method)
    ax1.plot(dfo['n_clusters'], dfo['silhouette'],label="silhouette")
    ax1.plot(df['n_clusters'], df['silhouette'],label="silhouette "+method)
    ax1.set_title("EM Performance evaluation %s"%data['name'])
    ax1.set_xlabel("Number of clusters")
    ax1.set_ylabel("Scores")
    ax1.legend()

    ax2.plot(dfo['n_clusters'], dfo['bic'],label="bic")
    ax2.plot(df['n_clusters'], df['bic'],label="bic "+method)
    ax2.set_title("EM BIC %s"%data['name'])
    ax2.set_xlabel("Number of clusters")
    ax2.set_ylabel("Scores")
    ax2.legend()


    fig, ax3 = plt.subplots(1,1)
    ax3.plot(df['n_clusters'], df['time'], label="time")
    ax3.plot(dfo['n_clusters'], dfo['time'], label="time "+method)
    ax3.set_title("EM Time %s"%data['name'])
    ax3.legend()

    plt.show()

    if not ncluster:
        ncluster = data['n_targets']
        
    perfo = dfo[df['n_clusters']==ncluster]
    perf = df[df['n_clusters']==ncluster]

    print(perfo)
    print("------")
    print(perf)